# Classifying Sequences

In this exercise, you will get familiar with how to build RNNs in Keras. You will build a recurrent model to classify moview reviews as either positive or negative.

In [2]:
%matplotlib inline

import numpy as np
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.layers import LSTM, SimpleRNN, GRU
from keras.datasets import imdb

Using TensorFlow backend.


## IMDB Sentiment Dataset

The large movie review dataset is a collection of 25k positive and 25k negative movie reviews from [IMDB](http://www.imdb.com). Here are some excerpts from the dataset, both easy and hard, to get a sense of why this dataset is challenging:

> Ah, I loved this movie.

> Quite honestly, The Omega Code is the worst movie I have seen in a very long time.

> The wit and pace and three show stopping Busby Berkley numbers put this ahead of the over-rated 42nd Street. 

> There simply was no suspense, precious little excitement and too many dull spots, most of them trying to show why "Nellie" (Monroe) was so messed up.

The dataset can be found at http://ai.stanford.edu/~amaas/data/sentiment/. Since this is a common dataset for RNNs, Keras has a preprocessed version built-in. The data is preprocessed by replacing words with indexes - review [Keras's docs](http://keras.io/datasets/#imdb-movie-reviews-sentiment-classification).

In [3]:
# We will limit to the most frequent 20k words defined by max_features, our vocabulary size
max_features = 20000
(X_train, y_train), (X_test, y_test) = imdb.load_data(nb_words=max_features)
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

In [4]:
print 'Representation of the first review:'
print X_train[0]

Representation of the first review:
[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


#### Exercise 1 - prepare the data

The reviews are different lengths but we need to fit them into a matrix to feed to Keras. We will do this by picking a maximum word length and cutting off words from the examples that are over that limit and padding the examples with 0 if they are under the limit.

Refer to the [Keras docs](http://keras.io/preprocessing/sequence/#pad_sequences) for the `pad_sequences` function. Use `pad_sequences` to prepare both `X_train` and `X_test` to be `maxlen` long at the most.

In [5]:
# Pad and clip the example sequences

maxlen = 80
X_train = sequence.pad_sequences(X_train, maxlen=maxlen))
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)

print 'X_train shape:', X_train.shape
print X_train[0]
print 'X_test shape:', X_test.shape
print X_test[0]

SyntaxError: invalid syntax (<ipython-input-5-7c5fa19d8b87>, line 4)

#### Exercise 2 - build an RNN for classifying reviews as positive or negative

Build a single-layer RNN model and train it. You will need to include these parts:

* An `Embedding` layer for efficiently one-hot encoding the inputs - [docs]()
* A recurrent layer. Keras has a [few variants](http://keras.io/layers/recurrent/) you could use. LSTM layers are by far the most popular for RNNs.
* A `Dense` layer for the hidden to output connection.
* A softmax to produce the final prediction.

You will need to decide how large your hidden state will be. You may also consider using some dropout on your recurrent or embedding layers - refers to docs for how to do this.

Training for longer will be much better overall, but since RNNs are expensive to train, you can use 1 epoch to test. You should be able to get > 70% accuracy with 1 epoch. How high can you get?

In [ ]:
# Design an recurrent model
model = Sequential()
model.add(Embedding(max_features, 32, input_length=maxlen))
model.add(LSTM(32))
model.add(Dense(32, activation='relu'))
model.add(Dense(2,activation='softmax'))

# The Adam optimizer can automatically adjust learning rates for you
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, batch_size=32, nb_epoch=2, validation_data=(X_test, y_test))
score, acc = model.evaluate(X_test, y_test, batch_size=32)
print('Test score:', score)
print('Test accuracy:', acc)

In [ ]:
y_pred = model.predict_classes(X_test)
y_true = np.argmax(y_test, axis=1)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.figure(figsize=(5, 5), dpi=100)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
y_pred = model.predict_classes(X_test, verbose=0)
y_true = np.argmax(y_test, axis=1) # it was one-hot encoded

In [ ]:
# Confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_true, y_pred)
plot_confusion_matrix(cm, ['negative', 'positive'], title='Accuracy: %.2f / 100' % (acc * 100))